<a href="https://colab.research.google.com/github/F4ULTER/SDBurritoRec/blob/main/SanDiegoBurritoReccomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Preferences: Content-Based Filtering

In [ ]:
#@title <-- Load imports and functions!
from __future__ import print_function
import ast
import pandas as pd
import numpy as np
import time
from google.colab import files

from IPython.display import Javascript, display, clear_output
from sklearn.metrics import r2_score


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import IntSlider

import math

### Parse for Burrito Data Set
Importing Burrito Data Set

In [ ]:
attributes = []
sheet_url = 'https://docs.google.com/spreadsheets/d/18HkrklYz1bKpDLeL-kaMrGjAhUM6LeJMIACwEljCgaw/edit#gid=1703829449'
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
bD = pd.read_csv(url_1,sep=',')
bD.head()


bD['Location'] = bD['Location'].str.replace("'", "" )
bD['Location'] = bD['Location'].str.strip()

bD['Burrito'] = bD['Burrito'].str.replace("'", "" )
bD['Burrito'] = bD['Burrito'].str.strip()

ROpt_temp = bD.iloc[:,0]
ROpt_temp = ROpt_temp.tolist()
ROpt = []
for i in ROpt_temp:
  i_lower = i.lower()
  if i_lower not in (string.lower() for string in ROpt):
        ROpt.append(i)
ROpt.sort()

###Input Your Burrito Ratings
*look at the code if you want, but even I don't get it*

In [ ]:

output = widgets.Output()
bDIndex = []
sent1 = 1
style = {'description_width': 'initial'}
burritos_rated = pd.DataFrame()


dropdown1 = widgets.Dropdown(
    options= ROpt,
    value=None,
    description='Resturant:',
)

BOpt = []

#on drop down 1 change, update drop down 2
def on_change(change):

  if change['type'] == 'change' and change['name'] == 'value':

        thing = (change['new'])
        filt = (bD['Location'] == thing)
        BOpt_temp = (bD.loc[filt, 'Burrito'])
        BOpt = []
        for i in BOpt_temp:
          i_lower = i.lower()
          if i_lower not in (string.lower() for string in BOpt):
                BOpt.append(i)
        BOpt.sort()
        BOpt_copy = BOpt.copy()
        dropdown2.options = BOpt_copy



dropdown1.observe(on_change)
#I don't know why, and i don't want to know why, but if i remove it the code doesn't work so

display(dropdown1)

dropdown2 = widgets.Dropdown(
            options= BOpt,
            value= None,
            description='Burrito:',
        )

display(dropdown2)

slide = widgets.IntSlider(
    value=0,
    min=1,
    max=5,
    step=1,
    description='Rating:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(slide)

button = widgets.Button(description="Submit Burrito Rating")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
      #If there are multiple ratings, the program will average the rating into one, and ignores irrelevant data
      tempDF = bD.loc[(bD['Location'].str.lower() == dropdown1.value.lower()) & (bD['Burrito'].str.lower() == dropdown2.value.lower())]
      tempDF = tempDF.iloc[:, np.r_[9, 16:26]].mean(axis = 0, skipna=True)
      burritoRatingParse = slide.value
      tempDF['User Rating'] = burritoRatingParse
      global burritos_rated
      burritos_rated = burritos_rated.append([tempDF], ignore_index=True)
      print("Added " + dropdown2.value + " burrito from " + dropdown1.value + " to rating list! ")

button.on_click(on_button_clicked)





###homie if ur reading this, just know this is a complete and total headache, and that my code looks like spaghetti

Dropdown(description='Resturant:', options=('Albertacos', 'Albertos 623 N Escondido Blvd, Escondido, CA 92025'…

Dropdown(description='Burrito:', options=(), value=None)

IntSlider(value=1, continuous_update=False, description='Rating:', max=5, min=1)

Button(description='Submit Burrito Rating', style=ButtonStyle())

Output()

###View Your Burrito Ratings

In [ ]:
print(f"Your Ratings")
burritos_rated.head(12)

Your Ratings


,Cost,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,overall,User Rating
0,5.99,4.00,2.0,2.5,4.0,4.0,3.0,3.5,4.0,5.0,3.20,4.0
1,9.00,2.00,3.5,3.0,1.5,1.0,1.0,2.5,1.5,3.5,1.50,3.0
2,6.50,3.75,5.0,3.0,3.0,4.5,4.5,3.5,3.0,4.5,3.25,5.0


## Weighted Average of Burrito Properties

Using user data, we can create a "model" burrito to predict what qualities of a burrito you value the most. While this will not be your perfect burrito, it allows us to prioritize certain qualities over others.

In [ ]:
model_burrito = pd.DataFrame()
weighted_burrito = burritos_rated.copy()


x  = 0
weightCounter = 0


while x < len(weighted_burrito.index):
 weighted_burrito.iloc[x,1:11] = weighted_burrito.iloc[x,1:11].multiply((weighted_burrito.iloc[x,11]))
 weightCounter+=(weighted_burrito.iloc[x,11])
 x+=1

print("Burritos with applied weights")

display(weighted_burrito.head())


for (col) in weighted_burrito:
  model_burrito [col] = weighted_burrito[col]
  model_burrito [col] = float(weighted_burrito[col].sum(axis = 0)/weightCounter)

model_burrito['Cost'] = weighted_burrito['Cost'].mean(axis = 0)
model_burrito['User Rating'] = weighted_burrito['User Rating'].mean(axis = 0)
model_burrito = model_burrito.mean(axis = 0)


print("\n\nYour Model Burrito")
m_burrito = pd.DataFrame(model_burrito)
display(m_burrito.head(10).transpose())


Burritos with applied weights


,Cost,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,overall,User Rating
0,5.99,16.00,8.0,10.0,16.0,16.0,12.0,14.0,16.0,20.0,12.80,4.0
1,9.00,6.00,10.5,9.0,4.5,3.0,3.0,7.5,4.5,10.5,4.50,3.0
2,6.50,18.75,25.0,15.0,15.0,22.5,22.5,17.5,15.0,22.5,16.25,5.0




Your Model Burrito


,Cost,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap
0,7.163333,3.395833,3.625,2.833333,2.958333,3.458333,3.125,3.25,2.958333,4.416667


## Finding the Top Dimentions
Excluding price, what qualities of a burrito do you value the most ?

In [ ]:

MBFilter = model_burrito.copy()
MBFilter = MBFilter.iloc[1:11].sort_values(ascending=False)
MBFilter = MBFilter.iloc[0:3]
MBFilter = MBFilter.to_frame()
print("Your Top 3 Burrito Qualities (According to data)")

MBFilter = MBFilter.transpose()
MBFilter.head()


Your Top 3 Burrito Qualities (According to data)


,Wrap,Temp,Meat:filling
0,4.416667,3.625,3.458333


### Finding the Most Similar Burrito
Sorting through the data set to find the burritio for you !

In [ ]:
#creating a weighted data set to sort through the values
bD_temp = bD.copy()
bD_weighted = pd.DataFrame()

#print(bD_temoc['Synergy'])

bD_weighted['Location'] = (bD_temp['Location'].to_frame())
bD_weighted['Burrito'] = (bD_temp['Burrito'].to_frame())

for (col) in MBFilter:
  #print(bD_temp[col])
  bD_weighted[col] = (bD_temp[col].to_frame())
  bD_weighted[col] = bD_weighted[col].mul(float(MBFilter[col]))

bD_weighted['Point Score'] = bD_weighted.sum(axis=1)
bD_weighted = bD_weighted.sort_values('Point Score',ascending=False)
print("List Of Burritos Ranked by Point Score")
display(bD_weighted.head(10))
indexList = (bD_weighted.index[0:3])
#print(bD_weighted)

List Of Burritos Ranked by Point Score


<ipython-input-12-a61039e576c9>:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  bD_weighted['Point Score'] = bD_weighted.sum(axis=1)


,Location,Burrito,Wrap,Temp,Meat:filling,Point Score
315,Rigobertos Taco Shop La Jolla,California,22.083333,18.125,17.291667,57.5
135,Humbertos,Pollo california,22.083333,18.125,17.291667,57.5
288,Cancun Mexican & Seafood,California,22.083333,18.125,17.291667,57.5
59,Rigobertos Taco Shop,Adobada,22.083333,18.125,17.291667,57.5
69,California Burritos,Chile relleno and carnitas,22.083333,18.125,17.291667,57.5
386,Lolitas Taco Shop,California,22.083333,18.125,17.291667,57.5
136,Chili Peppers,Surf & Turf,22.083333,18.125,17.291667,57.5
388,La Posta de Acapulco,California,22.083333,18.125,17.291667,57.5
412,Taco Stand,California,22.083333,18.125,17.291667,57.5
133,El Zarape,Surf and Turf,22.083333,18.125,17.291667,57.5
